In [1]:
import pandas as pd
from pandasql import sqldf
pd.set_option('display.max_columns', None)
hoursdf=pd.read_csv('time-data/281-time-2-25-2024.tsv', sep='\t',skipinitialspace=True)
hoursdf
hoursdf["PIN"] = hoursdf["PIN"].astype("string")
student_pins=pd.read_csv('time-data/student_pins.tsv', sep='\t',skipinitialspace=True)
student_pins["PIN"] = student_pins["PIN"].astype("string")
student_pins
pysqldf = lambda q: sqldf(q, globals())
hoursdf = pysqldf("select hoursdf.*,student_pins.email, student_pins.name from hoursdf, student_pins where hoursdf.PIN = student_pins.PIN")

#hoursdf
#student_pins.dtypes
#hoursdf.dtypes
#hours_df2=hoursdf.join(student_pins,how="left",on=['PIN'],lsuffix="_left",rsuffix="_right")
#hours_df2

In [2]:
hoursdf.info()
hoursdf.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        864 non-null    object
 1   TYPE        864 non-null    object
 2   Unnamed: 2  0 non-null      object
 3   Unnamed: 3  0 non-null      object
 4   PIN         864 non-null    object
 5   email       864 non-null    object
 6   name        864 non-null    object
dtypes: object(7)
memory usage: 47.4+ KB


,Date,TYPE,Unnamed: 2,Unnamed: 3,PIN,email,name
0,1/13/2024 18:00:00,Out,None,None,Eagle,maximilian.geissler0@gmail.com,Maximilian Geissler
1,1/11/2024 20:04:41,Out,None,None,phoenix,anheitkamp.2023@gmail.com,Andrew Heitkamp
2,1/20/2024 9:02:17,in,None,None,phoenix,anheitkamp.2023@gmail.com,Andrew Heitkamp
3,1/18/2024 18:05:30,in,None,None,J.T.M.,jtmolfenter07@gmail.com,John Molfenter
4,1/8/2024 17:49:11,In,None,None,Ghost,hayley@nobudgetracing.com,Hayley Morton


In [3]:
hoursdf['tstamp'] = pd.to_datetime(hoursdf.Date)
hoursdf['shopdate'] = hoursdf.tstamp.dt.strftime('%m-%d, %a')
hoursdf['shopday'] = hoursdf.tstamp.dt.strftime('%a')

In [4]:
hoursdf

,Date,TYPE,Unnamed: 2,Unnamed: 3,PIN,email,name,tstamp,shopdate,shopday
0,1/13/2024 18:00:00,Out,None,None,Eagle,maximilian.geissler0@gmail.com,Maximilian Geissler,2024-01-13 18:00:00,"01-13, Sat",Sat
1,1/11/2024 20:04:41,Out,None,None,phoenix,anheitkamp.2023@gmail.com,Andrew Heitkamp,2024-01-11 20:04:41,"01-11, Thu",Thu
2,1/20/2024 9:02:17,in,None,None,phoenix,anheitkamp.2023@gmail.com,Andrew Heitkamp,2024-01-20 09:02:17,"01-20, Sat",Sat
3,1/18/2024 18:05:30,in,None,None,J.T.M.,jtmolfenter07@gmail.com,John Molfenter,2024-01-18 18:05:30,"01-18, Thu",Thu
4,1/8/2024 17:49:11,In,None,None,Ghost,hayley@nobudgetracing.com,Hayley Morton,2024-01-08 17:49:11,"01-08, Mon",Mon
...,...,...,...,...,...,...,...,...,...,...
859,2/24/2024 21:32:17,Out,None,None,JeffBagels,mmakhtoub1@gmail.com,Marwan Makhtoub,2024-02-24 21:32:17,"02-24, Sat",Sat
860,2/24/2024 22:27:23,Out,None,None,0120,AnaReyesRayos@gmail.com,Ana Reyes Rayos,2024-02-24 22:27:23,"02-24, Sat",Sat
861,2/24/2024 22:27:49,Out,None,None,1116,dhanyagan2009@gmail.com,dhanya Rajagopalan,2024-02-24 22:27:49,"02-24, Sat",Sat
862,2/24/2024 22:34:39,Out,None,None,JeffBagels,mmakhtoub1@gmail.com,Marwan Makhtoub,2024-02-24 22:34:39,"02-24, Sat",Sat


In [5]:
def shop_hours(s):
    seconds_this_day = (s.max() - s.min()).seconds/60
    minutes_this_day = seconds_this_day / 60
    if minutes_this_day == 0:
        minutes_this_day = 1.0
    return minutes_this_day
attendance = hoursdf.groupby(['name','shopdate']).agg( shop_hours=('tstamp',shop_hours))
attendance.reset_index()


,name,shopdate,shop_hours
0,Aditya Harikumar,"01-08, Mon",2.051667
1,Aditya Harikumar,"01-11, Thu",2.032500
2,Aditya Harikumar,"01-13, Sat",3.205278
3,Aditya Harikumar,"01-15, Mon",4.192778
4,Aditya Harikumar,"01-17, Wed",2.253333
...,...,...,...
428,dhanya Rajagopalan,"02-15, Thu",2.028889
429,dhanya Rajagopalan,"02-17, Sat",12.757500
430,dhanya Rajagopalan,"02-21, Wed",1.547778
431,dhanya Rajagopalan,"02-22, Thu",1.595556


In [6]:
timesheet = attendance.pivot_table(index='name',columns='shopdate',values='shop_hours',fill_value=0,aggfunc='sum')
timesheet

shopdate,"01-08, Mon","01-10, Wed","01-11, Thu","01-13, Sat","01-15, Mon","01-17, Wed","01-18, Thu","01-20, Sat","01-22, Mon","01-24, Wed","01-25, Thu","01-27, Sat","01-29, Mon","01-31, Wed","02-01, Thu","02-03, Sat","02-04, Sun","02-05, Mon","02-07, Wed","02-08, Thu","02-10, Sat","02-12, Mon","02-13, Tue","02-14, Wed","02-15, Thu","02-17, Sat","02-18, Sun","02-19, Mon","02-20, Tue","02-21, Wed","02-22, Thu","02-24, Sat"
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aditya Harikumar,2.051667,0.000000,2.032500,3.205278,4.192778,2.253333,1.962222,2.039722,2.084722,1.815833,2.001944,0.000000,2.371944,1.951111,0.000000,4.715556,0.000000,1.397222,2.146111,1.512222,2.606667,2.056389,2.372222,1.483056,1.750000,0.000000,0.000000,2.191389,0.000000,0.000000,0.000000,0.000000
Albert Mathisz,1.883056,1.835000,1.959444,0.000000,2.044167,2.065278,0.000000,0.000000,2.458889,1.930833,0.000000,0.000000,2.142500,1.876389,0.000000,3.819167,0.000000,1.000000,0.000000,0.000000,0.000000,1.915278,0.000000,1.988611,0.000000,3.138056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Ana Reyes Rayos,2.098889,0.000000,0.000000,3.187500,3.848889,2.153611,2.040000,4.373333,0.000000,2.108056,0.000000,0.000000,0.000000,1.945000,1.790000,6.756944,0.000000,2.177500,2.180278,1.985000,6.126944,2.166111,1.959167,2.016667,2.190278,12.717500,4.633611,2.196389,1.947222,0.000000,0.000000,13.230556
Andrew Heitkamp,2.027778,1.820000,1.968056,9.009722,5.112500,1.000000,0.000000,9.000000,2.787500,2.080556,1.667222,6.890556,1.000000,1.908056,1.000000,8.511667,0.000000,0.000000,2.230000,1.697222,9.199444,2.308889,0.000000,1.753889,2.229167,1.000000,0.000000,1.000000,0.000000,5.390556,1.859167,11.922222
Antony Manavalan,1.537778,2.200833,2.078333,7.215556,5.630000,2.353611,2.089444,8.386667,1.611111,2.214444,1.958889,6.171389,0.000000,0.000000,2.289167,0.000000,0.000000,2.459167,2.201667,0.648889,4.990278,2.134722,0.000000,0.000000,2.318056,0.000000,0.000000,2.104722,0.000000,0.000000,2.614167,9.557222
Ayush Ghosh,1.946111,2.179722,0.000000,7.330833,2.962500,2.118056,0.000000,4.605278,2.033889,0.000000,2.207222,7.208611,0.000000,0.000000,2.125833,8.168056,0.000000,0.000000,2.243333,1.962500,4.868889,2.197222,0.000000,1.900556,0.000000,10.273333,0.000000,2.251944,0.000000,2.053889,0.000000,3.771667
Benjamin Cowden,2.273889,1.510556,2.074722,8.851111,3.245556,2.657222,2.039167,7.852778,2.117500,0.000000,1.994444,9.201667,2.295000,1.043056,2.555833,7.939167,0.572222,2.108611,0.000000,2.225556,7.324722,1.000000,1.672500,0.000000,2.427778,10.166111,0.000000,2.189722,1.818611,2.006667,2.225556,9.325833
Benjamin Jones,2.160833,0.000000,1.997778,8.952222,5.929444,0.000000,2.104444,8.943333,2.208056,0.000000,2.122500,9.167500,2.271111,0.000000,2.084444,8.950278,0.000000,0.000000,0.000000,2.084444,9.101111,2.088611,2.220000,0.000000,0.000000,0.000000,0.000000,2.114444,0.000000,0.000000,2.296111,7.970278
Eric Shi,2.017500,1.000000,0.000000,8.721389,3.476667,1.000000,0.000000,8.738611,1.000000,1.855833,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,8.828889,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000


In [7]:

timesheet.loc['AVAILABLE'] = [2,2,2,9,  2,2,2,9,  2,2,2,9,    2,2,2,9,0,  2,2,2,9,  2,0,2,2,9,    0,    2, 0, 2, 2, 9 ]
total_available_hours = timesheet.loc['AVAILABLE'].sum()
timesheet['student_total'] = timesheet.sum(axis=1)


#shop_available_hours = pd.DataFrame({ 
#    "date": [ "01/08", "01/10", "01/11", "01/13" ],
#    "hours": [ 2, 2, 2, 9]
#})

timesheet['student_percent'] = timesheet['student_total'] / total_available_hours
timesheet['student_percent_nice'] = timesheet['student_percent'].map('{:.2%}'.format)
timesheet.drop(columns='student_percent')

timesheet


shopdate,"01-08, Mon","01-10, Wed","01-11, Thu","01-13, Sat","01-15, Mon","01-17, Wed","01-18, Thu","01-20, Sat","01-22, Mon","01-24, Wed","01-25, Thu","01-27, Sat","01-29, Mon","01-31, Wed","02-01, Thu","02-03, Sat","02-04, Sun","02-05, Mon","02-07, Wed","02-08, Thu","02-10, Sat","02-12, Mon","02-13, Tue","02-14, Wed","02-15, Thu","02-17, Sat","02-18, Sun","02-19, Mon","02-20, Tue","02-21, Wed","02-22, Thu","02-24, Sat",student_total,student_percent,student_percent_nice
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aditya Harikumar,2.051667,0.000000,2.032500,3.205278,4.192778,2.253333,1.962222,2.039722,2.084722,1.815833,2.001944,0.000000,2.371944,1.951111,0.000000,4.715556,0.000000,1.397222,2.146111,1.512222,2.606667,2.056389,2.372222,1.483056,1.750000,0.000000,0.000000,2.191389,0.000000,0.000000,0.000000,0.000000,50.193889,0.478037,47.80%
Albert Mathisz,1.883056,1.835000,1.959444,0.000000,2.044167,2.065278,0.000000,0.000000,2.458889,1.930833,0.000000,0.000000,2.142500,1.876389,0.000000,3.819167,0.000000,1.000000,0.000000,0.000000,0.000000,1.915278,0.000000,1.988611,0.000000,3.138056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.056667,0.286254,28.63%
Ana Reyes Rayos,2.098889,0.000000,0.000000,3.187500,3.848889,2.153611,2.040000,4.373333,0.000000,2.108056,0.000000,0.000000,0.000000,1.945000,1.790000,6.756944,0.000000,2.177500,2.180278,1.985000,6.126944,2.166111,1.959167,2.016667,2.190278,12.717500,4.633611,2.196389,1.947222,0.000000,0.000000,13.230556,85.829444,0.817423,81.74%
Andrew Heitkamp,2.027778,1.820000,1.968056,9.009722,5.112500,1.000000,0.000000,9.000000,2.787500,2.080556,1.667222,6.890556,1.000000,1.908056,1.000000,8.511667,0.000000,0.000000,2.230000,1.697222,9.199444,2.308889,0.000000,1.753889,2.229167,1.000000,0.000000,1.000000,0.000000,5.390556,1.859167,11.922222,96.374167,0.917849,91.78%
Antony Manavalan,1.537778,2.200833,2.078333,7.215556,5.630000,2.353611,2.089444,8.386667,1.611111,2.214444,1.958889,6.171389,0.000000,0.000000,2.289167,0.000000,0.000000,2.459167,2.201667,0.648889,4.990278,2.134722,0.000000,0.000000,2.318056,0.000000,0.000000,2.104722,0.000000,0.000000,2.614167,9.557222,74.766111,0.712058,71.21%
Ayush Ghosh,1.946111,2.179722,0.000000,7.330833,2.962500,2.118056,0.000000,4.605278,2.033889,0.000000,2.207222,7.208611,0.000000,0.000000,2.125833,8.168056,0.000000,0.000000,2.243333,1.962500,4.868889,2.197222,0.000000,1.900556,0.000000,10.273333,0.000000,2.251944,0.000000,2.053889,0.000000,3.771667,74.409444,0.708661,70.87%
Benjamin Cowden,2.273889,1.510556,2.074722,8.851111,3.245556,2.657222,2.039167,7.852778,2.117500,0.000000,1.994444,9.201667,2.295000,1.043056,2.555833,7.939167,0.572222,2.108611,0.000000,2.225556,7.324722,1.000000,1.672500,0.000000,2.427778,10.166111,0.000000,2.189722,1.818611,2.006667,2.225556,9.325833,102.715556,0.978243,97.82%
Benjamin Jones,2.160833,0.000000,1.997778,8.952222,5.929444,0.000000,2.104444,8.943333,2.208056,0.000000,2.122500,9.167500,2.271111,0.000000,2.084444,8.950278,0.000000,0.000000,0.000000,2.084444,9.101111,2.088611,2.220000,0.000000,0.000000,0.000000,0.000000,2.114444,0.000000,0.000000,2.296111,7.970278,84.766944,0.807304,80.73%
Eric Shi,2.017500,1.000000,0.000000,8.721389,3.476667,1.000000,0.000000,8.738611,1.000000,1.855833,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,8.828889,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,43.638889,0.415608,41.56%


In [8]:
timesheet=timesheet.sort_values(by=['student_total'],ascending=False)
timesheet[['student_percent_nice']]






shopdate,student_percent_nice
name,
Hayley Morton,121.54%
AVAILABLE,100.00%
Benjamin Cowden,97.82%
Andrew Heitkamp,91.78%
John Molfenter,89.03%
dhanya Rajagopalan,85.11%
Ana Reyes Rayos,81.74%
Benjamin Jones,80.73%
Victor Bellmore,76.66%


In [9]:
def color_code(val):
    
    if val == 1.0:
        return "background-color: yellow;color:black;opacity:40%"
    elif val == 0:
        return "background-color: red;color:white;opacity:40%"
    else:
        return 'color: green;font-weight:bold'

timesheet=timesheet.transpose()
timesheet.style.format(precision=3).map(color_code)
#timesheet.style.background_gradient(cmap="Blues")
#timesheet.style.highlight_max()


name,Hayley Morton,AVAILABLE,Benjamin Cowden,Andrew Heitkamp,John Molfenter,dhanya Rajagopalan,Ana Reyes Rayos,Benjamin Jones,Victor Bellmore,Sadra Saharkhiz,Antony Manavalan,Ayush Ghosh,Trevor,Marwan Makhtoub,Sophie Weaver,Maximilian Geissler,Aditya Harikumar,Isaac Boroughs,Eric Shi,Sanjith kalluri,Albert Mathisz,Nathan Wicker,Pranav Kata,Joshua Spurck
shopdate,,,,,,,,,,,,,,,,,,,,,,,,
"01-08, Mon",2.369,2.000,2.274,2.028,1.955,2.245,2.099,2.161,2.130,1.048,1.538,1.946,2.031,1.945,2.095,2.103,2.052,2.123,2.018,1.898,1.883,2.057,1.000,2.033
"01-10, Wed",2.487,2.000,1.511,1.820,0.996,1.482,0.000,0.000,0.000,1.814,2.201,2.180,1.995,0.000,0.000,1.985,0.000,0.000,1.000,0.000,1.835,1.000,1.000,0.000
"01-11, Thu",2.734,2.000,2.075,1.968,0.000,0.970,0.000,1.998,0.000,1.646,2.078,0.000,2.026,2.381,0.000,0.789,2.033,1.000,0.000,0.000,1.959,0.000,0.000,0.000
"01-13, Sat",8.751,9.000,8.851,9.010,0.000,5.466,3.188,8.952,7.963,5.606,7.216,7.331,5.175,0.000,7.809,8.216,3.205,4.128,8.721,0.000,0.000,2.628,6.543,0.000
"01-15, Mon",6.032,2.000,3.246,5.112,5.730,5.759,3.849,5.929,3.542,5.919,5.630,2.962,0.000,4.333,0.000,6.555,4.193,4.013,3.477,3.732,2.044,0.000,0.000,0.000
"01-17, Wed",5.309,2.000,2.657,1.000,2.180,2.157,2.154,0.000,0.000,2.153,2.354,2.118,0.000,0.000,0.000,0.000,2.253,0.000,1.000,0.000,2.065,0.000,0.000,0.000
"01-18, Thu",2.579,2.000,2.039,0.000,2.000,1.911,2.040,2.104,0.000,1.460,2.089,0.000,0.000,0.000,0.000,0.736,1.962,2.083,0.000,1.398,0.000,0.000,0.000,0.000
"01-20, Sat",1.699,9.000,7.853,9.000,6.490,8.238,4.373,8.943,7.986,4.444,8.387,4.605,4.718,2.663,6.757,1.797,2.040,3.586,8.739,0.000,0.000,0.000,0.000,0.000
"01-22, Mon",2.912,2.000,2.118,2.788,1.654,2.066,0.000,2.208,2.127,2.026,1.611,2.034,2.170,2.613,0.000,1.000,2.085,2.237,1.000,1.863,2.459,1.000,0.000,0.000
